In [1]:
import pandas as pd
import numpy as np
import re
import builtins

In [2]:
%load_ext gams.magic
m = gams.exchange_container

In [3]:
gams.reset()
m = gams.exchange_container
df = pd.read_csv('FInalFoodData.csv', index_col="Item")
df 

,Description,Brand,Calories,Serving Size,Nutrients,Nov 2023 Price in Dollars,Percent Change from Nov 2022,Percent Change from Oct 2023
Item,,,,,,,,
chicken,CHICKEN,Keystone Brand Meats Inc,107.0 kcal,56.0 g,"Fatty acids, total trans: 0.0 g, Carbohydrate,...",0.12/oz,3.1,2.6
yogurt,YOGURT,Agri-Service LLC,94.0 kcal,170.0 ml,"Iron, Fe: 0.0 mg, Calcium, Ca: 118.0 mg, Fiber...",0.07/oz,NaN,NaN
cereal,CEREAL,"General Mills, Inc.",387.0 kcal,31.0 g,"Fatty acids, total polyunsaturated: 1.61 g, Ir...",0.27/oz,NaN,NaN
Corn,CORN,"Kcb Industries, L.L.C.",545.0 kcal,33.0 g,"Vitamin D (D2 + D3), International Units: 0.0 ...",0.17/oz,NaN,NaN
Potatoes,POTATOES,THE LITTLE POTATO COMPANY,74.0 kcal,148.0 g,"Iron, Fe: 0.73 mg, Cholesterol: 0.0 mg, Calciu...",0.05/oz,2.7,-2.8
...,...,...,...,...,...,...,...,...
Ice cream,ICE CREAM,"Clover Stornetta Farms, Inc.",205.0 kcal,88.0 g,"Energy: 205.0 kcal, Carbohydrate, by differenc...",0.17/oz,NaN,NaN
cookies,COOKIES,KNO,441.0 kcal,100.0 g,"Fiber, total dietary: 0.0 g, Sodium, Na: 0.0 m...",0.20/oz,NaN,NaN
brownies,BROWNIES,TWO-BITE,460.0 kcal,50.0 g,"Iron, Fe: 2.88 mg, Fiber, total dietary: 0.0 g...",0.34/oz,NaN,NaN


In [4]:
df['Calories'] = df['Calories'].str.extract('(\d+\.\d+)').astype(float)

In [5]:
df['Serving Size'] = df['Serving Size'].str.extract('(\d+\.\d+)').astype(float)

In [6]:
# Extract the numeric part of the 'Nov 2023 Price in Dollars' column and convert to float
df['Price'] = df['Nov 2023 Price in Dollars'].str.extract(r'(\d+\.\d+)').astype(float)

# Drop the original 'Nov 2023 Price in Dollars' column
df.drop('Nov 2023 Price in Dollars', axis=1, inplace=True)


In [7]:
def extract_nutrients(row):
    nutrients = {}
    items = re.split(', ', row['Nutrients'])
    for item in items:
        match = re.match(r'(.+?): (.+?) (.+)', item)
        if match:
            nutrient, value = match.groups()[:2]
            nutrients[nutrient] = float(value) if '.' in value else int(value)
    return pd.Series(nutrients)

# Apply the function to each row and concatenate the result with the original DataFrame
df = pd.concat([df, df.apply(extract_nutrients, axis=1)], axis=1)

# Drop the original 'Nutrients' column
df = df.drop('Nutrients', axis=1)

In [8]:
df.head()

,Description,Brand,Calories,Serving Size,Percent Change from Nov 2022,Percent Change from Oct 2023,Price,Alanine,Arginine,Ash,...,other,soluble,total,total ascorbic acid,total dietary,total including NLEA,total monounsaturated,total polyunsaturated,total saturated,total trans
Item,,,,,,,,,,,,,,,,,,,,,
chicken,CHICKEN,Keystone Brand Meats Inc,107.0,56.0,3.1,2.6,0.12,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,NaN,0.89,0.0
yogurt,YOGURT,Agri-Service LLC,94.0,170.0,NaN,NaN,0.07,NaN,NaN,NaN,...,NaN,NaN,NaN,0.7,0.0,15.88,NaN,NaN,1.76,NaN
cereal,CEREAL,"General Mills, Inc.",387.0,31.0,NaN,NaN,0.27,NaN,NaN,NaN,...,48.0,NaN,NaN,19.4,6.5,29.03,1.61,1.61,0.00,0.0
Corn,CORN,"Kcb Industries, L.L.C.",545.0,33.0,NaN,NaN,0.17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.1,24.24,NaN,NaN,4.55,0.0
Potatoes,POTATOES,THE LITTLE POTATO COMPANY,74.0,148.0,2.7,-2.8,0.05,NaN,NaN,NaN,...,NaN,NaN,NaN,2.4,2.0,0.68,NaN,NaN,0.00,0.0


In [9]:
df.rename(columns={'by difference': 'Carbs'}, inplace=True)
df.rename(columns={'Total lipid (fat)': 'Fat'}, inplace=True)
df.rename(columns={'Ca': 'Calcium'}, inplace=True)
df.rename(columns={'Vitamin D (D2 + D3)': 'Vitamin D'}, inplace=True)
df.rename(columns={'Serving Size': 'Serving'}, inplace=True)

In [10]:
df = df[["Calories", "Serving", 'Protein', "Carbs", "Fat", "Calcium", "Price"]]
df

,Calories,Serving,Protein,Carbs,Fat,Calcium,Price
Item,,,,,,,
chicken,107.0,56.0,21.43,0.00,1.79,0.0,0.12
yogurt,94.0,170.0,2.94,15.88,2.94,118.0,0.07
cereal,387.0,31.0,6.45,83.87,3.23,323.0,0.27
Corn,545.0,33.0,6.06,54.55,33.33,61.0,0.17
Potatoes,74.0,148.0,2.70,16.89,0.00,0.0,0.05
...,...,...,...,...,...,...,...
Ice cream,205.0,88.0,2.27,25.00,10.23,68.0,0.17
cookies,441.0,100.0,7.37,68.37,13.26,NaN,0.20
brownies,460.0,50.0,2.00,56.00,26.00,0.0,0.34


In [11]:
df_limited = df.head(10)

In [12]:
df.fillna(0, inplace=True)

In [13]:
df = df.drop(['Rice Krispies', 'chili', 'Cornbread', "Instant oatmeal", 'Cheese curds', 'Peanut butter and jelly sandwiches'])
# df = df.drop(['chili'])
# df = df.drop(['Cornbread', "Instant oatmeal"])
# df = df.drop(['Peanut butter and jelly sandwiches'])

In [14]:
foods = df.index.values.tolist()
foods

['chicken',
 'yogurt',
 'cereal',
 'Corn',
 'Potatoes',
 'Green beans',
 'Carrots',
 'Tomatoes',
 'Bell peppers',
 'Onions',
 'Lettuce',
 'Spinach',
 'Broccoli',
 'Cauliflower',
 'Cucumbers',
 'Zucchini',
 'Peas',
 'Brussels sprouts',
 'Apples',
 'Bananas',
 'Grapes',
 'Oranges',
 'Strawberries',
 'Blueberries',
 'Watermelon',
 'Pears',
 'Cherries',
 'Plums',
 'Cantaloupe',
 'Raspberries',
 'Blackberries',
 'Kiwi',
 'Popcorn',
 'Pretzels',
 'Trail mix',
 'Beef jerky',
 'Potato chips',
 'Granola bars',
 'Applesauce',
 'peanuts',
 'almonds',
 'Crackers',
 'Cheese slices',
 'Fruit cups',
 'marshmallows',
 'chocolate',
 'graham crackers',
 'Milk',
 'Cheese',
 'steaks',
 'burgers',
 'Pork (bacon, ham)',
 'Eggs',
 'Wheat bread',
 'Oatmeal',
 'Pancake mix',
 'Pasta',
 'Rice',
 'Peaches',
 'Instant noodles',
 'Frozen pizza',
 'mac and cheese',
 'Canned soups',
 'cream cheese',
 'Chips and salsa',
 'Fruit snacks',
 'dried fruits',
 'String cheese',
 'Protein bar',
 'Coffee',
 'tea',
 'Energy dr

In [15]:
meats = ['chicken', 'steaks', 'burgers', 'Pork (bacon, ham)']
fruits = ['Apples','Bananas','Grapes','Oranges','Strawberries','Blueberries','Watermelon','Pears','Cherries','Plums','Cantaloupe','Raspberries','Blackberries','Kiwi','Peaches']  
vegetables = ['Corn','Potatoes','Green beans','Carrots','Tomatoes','Bell peppers','Onions','Lettuce','Spinach','Broccoli','Cauliflower','Cucumbers','Zucchini','Peas','Brussels sprouts','Salads']        
breakfast = ['yogurt','cereal', 'Eggs', 'Oatmeal','Pancake mix', 'cream cheese','Toaster pastries','Frozen waffles']  
frozen = ['Frozen pizza', 'Pizza rolls','mozzarella sticks','frozen vegetables']
drinks = ['Coffee','tea','Energy drinks','sports drinks','Fruit juices','smoothies','Soda']
desserts = ['Ice cream','cookies','brownies', 'marshmallows','chocolate'] 
snacks = ['Popcorn','Pretzels','Trail mix','Beef jerky','Potato chips','Granola bars','Applesauce','peanuts','almonds','Crackers','Cheese slices','Fruit cups','graham crackers', "Hummus",'Chips and salsa','Fruit snacks','dried fruits',"String cheese",'Protein bar']
dry = ["pita bread", 'Instant noodles', "mac and cheese", 'Pasta','Rice', 'Wheat bread','Canned soups']   
dairy = ['Cheese slices', 'Milk','Cheese',]
    

In [16]:
len(meats) + len(fruits) + len(vegetables) + len(breakfast) + len(frozen) + len(drinks) + len(desserts) + len(snacks) + len(dry)

85

In [17]:
len(df)


87

In [18]:
leftovers = []
for i in foods:
    if i in meats:
        continue
    if i in fruits:
        continue
    if i in vegetables:
        continue
    if i in breakfast:
        continue
    if i in frozen:
        continue
    if i in drinks:
        continue
    if i in desserts:
        continue
    if i in snacks:
        continue
    if i in dry:
        continue
    if i in dairy:
        continue
    
leftovers

[]

In [19]:
import pandas as pd

# Assuming df is your original DataFrame
# Assuming meats, fruits, vegetables, etc. are lists containing food items of each type

# Create an empty dictionary to store DataFrames for each type of food
food_dict = {}

# Iterate through the food items in the original DataFrame
for i in df.index.values:
    if i in meats:
        if 'Meats' not in food_dict:
            food_dict['Meats'] = pd.DataFrame(columns=df.columns)
        food_dict['Meats'] = pd.concat([food_dict['Meats'], df.loc[[i]]])
    elif i in fruits:
        if 'Fruits' not in food_dict:
            food_dict['Fruits'] = pd.DataFrame(columns=df.columns)
        food_dict['Fruits'] = pd.concat([food_dict['Fruits'], df.loc[[i]]])
    elif i in vegetables:
        if 'Vegetables' not in food_dict:
            food_dict['Vegetables'] = pd.DataFrame(columns=df.columns)
        food_dict['Vegetables'] = pd.concat([food_dict['Vegetables'], df.loc[[i]]])
    elif i in breakfast:
        if 'Breakfast' not in food_dict:
            food_dict['Breakfast'] = pd.DataFrame(columns=df.columns)
        food_dict['Breakfast'] = pd.concat([food_dict['Breakfast'], df.loc[[i]]])
    elif i in frozen:
        if 'Frozen' not in food_dict:
            food_dict['Frozen'] = pd.DataFrame(columns=df.columns)
        food_dict['Frozen'] = pd.concat([food_dict['Frozen'], df.loc[[i]]])
    elif i in drinks:
        if 'Drinks' not in food_dict:
            food_dict['Drinks'] = pd.DataFrame(columns=df.columns)
        food_dict['Drinks'] = pd.concat([food_dict['Drinks'], df.loc[[i]]])
    elif i in desserts:
        if 'Desserts' not in food_dict:
            food_dict['Desserts'] = pd.DataFrame(columns=df.columns)
        food_dict['Desserts'] = pd.concat([food_dict['Desserts'], df.loc[[i]]])
    elif i in snacks:
        if 'Snacks' not in food_dict:
            food_dict['Snacks'] = pd.DataFrame(columns=df.columns)
        food_dict['Snacks'] = pd.concat([food_dict['Snacks'], df.loc[[i]]])
    elif i in dry:
        if 'Dry' not in food_dict:
            food_dict['Dry'] = pd.DataFrame(columns=df.columns)
        food_dict['Dry'] = pd.concat([food_dict['Dry'], df.loc[[i]]])
    elif i in dairy:
        if 'Dairy' not in food_dict:
            food_dict['Dairy'] = pd.DataFrame(columns=df.columns)
        food_dict['Dairy'] = pd.concat([food_dict['Dairy'], df.loc[[i]]])

# Now, the food_dict dictionary contains DataFrames for each type of food
# Access them like food_dict['Meats'], food_dict['Fruits'], etc.

# Function to fill missing calorie values with the average of the category
def fill_missing_calories(food_category_df):
    # Calculate the average calories, excluding zero values
    avg_calories = food_category_df.loc[food_category_df['Calories'] > 0, 'Calories'].mean()
    
    # If there are no valid calorie values, avoid division by zero
    if np.isnan(avg_calories):
        avg_calories = 0

    # Fill missing calorie values with the average
    food_category_df.loc[food_category_df['Calories'] <= 0, 'Calories'] = avg_calories
    return food_category_df

# Apply the function to each category in food_dict
for category in food_dict:
    food_dict[category] = fill_missing_calories(food_dict[category])
    
for category in food_dict:
    if 'Calories' in food_dict[category].columns:
        food_dict[category]['Calories'] = food_dict[category]['Calories'].round(1)



In [20]:
food_dict

{'Meats':                    Calories  Serving  Protein  Carbs    Fat  Calcium  Price
 chicken               107.0     56.0    21.43   0.00   1.79      0.0   0.12
 steaks                179.5      0.0    11.90   4.34   9.12     17.0   0.68
 burgers               252.0    151.0    17.22   0.00  19.87     20.0   0.40
 Pork (bacon, ham)     179.5      0.0     6.26  10.40   8.27     21.0   0.28,
 'Breakfast':                   Calories  Serving  Protein  Carbs    Fat  Calcium  Price
 yogurt                94.0    170.0     2.94  15.88   2.94    118.0   0.07
 cereal               387.0     31.0     6.45  83.87   3.23    323.0   0.27
 Eggs                 143.0     63.0    12.70   0.00   9.52     63.0   0.16
 Oatmeal              378.0     45.0    11.11  68.89   6.67     44.0   0.11
 Pancake mix          333.0     36.0     5.56  75.00   0.00      0.0   0.07
 cream cheese         321.0     28.0     7.14   7.14  32.14     71.0   0.17
 Toaster pastries     375.0    104.0     2.88  71.15   8.65 

In [21]:
meats_df = food_dict["Meats"]
fruits_df = food_dict["Fruits"] 
vegetables_df = food_dict["Vegetables"]
breakfast_df = food_dict['Breakfast'] 
frozen_df = food_dict["Frozen"]
drinks_df = food_dict["Drinks"]
desserts_df = food_dict["Desserts"]
snacks_df = food_dict["Snacks"]
dry_df = food_dict["Dry"]   
dairy_df = food_dict["Dairy"]

In [22]:
dairy_df
snacks_df

,Calories,Serving,Protein,Carbs,Fat,Calcium,Price
Popcorn,373.0,51.0,11.76000,72.55000,3.92,0.0,0.79
Pretzels,464.0,28.0,7.14000,64.29000,21.43,0.0,0.14
Trail mix,439.0,57.0,12.28000,52.63000,24.56,70.0,0.24
Beef jerky,500.1,0.0,33.20000,11.00000,25.60,20.0,0.55
Potato chips,530.0,0.0,3.53000,45.94000,35.34,0.0,0.59
Granola bars,429.0,35.0,5.71000,65.71000,14.29,57.0,0.17
Applesauce,79.0,126.0,0.00000,21.43000,0.00,0.0,0.07
peanuts,571.0,28.0,21.43000,21.43000,46.43,0.0,0.15
almonds,633.0,30.0,23.33000,13.33000,56.67,333.0,0.38
Crackers,500.0,14.0,14.29000,64.29000,21.43,71.0,0.19


In [23]:
# Item = m.addSet('Item',records=df_limited.index)
# p = m.addParameter('price',[Item],records=df_limited[["Price"]].to_numpy())
# cal = m.addParameter('calories',[Item],records=df_limited[["Calories"]].to_numpy())
# pro = m.addParameter('protein',[Item],records=df_limited[["Protein"]].to_numpy())
# carbs = m.addParameter('carbs',[Item],records=df_limited[["Carbs"]].to_numpy())
# fat = m.addParameter('fat',[Item],records=df_limited[["Fat"]].to_numpy())
# calcium = m.addParameter('calcium',[Item],records=df_limited[["Calcium"]].to_numpy())
# display(calcium.records)

In [24]:
Item = m.addSet('Item',records=df.index)
me = m.addSet("meat",domain = [Item],records=meats_df.index)
fruit = m.addSet("fruit",domain = [Item],records=fruits_df.index)
veg = m.addSet("vegetable",domain = [Item],records=vegetables_df.index)
bf = m.addSet("breakfast",domain = [Item],records=breakfast_df.index)
fr = m.addSet("frozen",domain = [Item],records=frozen_df.index)
dr = m.addSet("drink",domain = [Item],records=drinks_df.index)
de = m.addSet("dessert",domain = [Item],records=desserts_df.index)
dry = m.addSet("dry",domain = [Item],records=dry_df.index)
snack = m.addSet("snack",domain = [Item],records=snacks_df.index)
dairy = m.addSet("dairy",domain = [Item],records=dairy_df.index)
p = m.addParameter('price',[Item],records=df[["Price"]].to_numpy())
cal = m.addParameter('calories',[Item],records=df[["Calories"]].to_numpy())
pro = m.addParameter('protein',[Item],records=df[["Protein"]].to_numpy())
carbs = m.addParameter('carbs',[Item],records=df[["Carbs"]].to_numpy())
fat = m.addParameter('fat',[Item],records=df[["Fat"]].to_numpy())
calcium = m.addParameter('calcium',[Item],records=df[["Calcium"]].to_numpy())
display(bf.records)

,Item,element_text
0,yogurt,
1,cereal,
2,Eggs,
3,Oatmeal,
4,Pancake mix,
5,cream cheese,
6,Toaster pastries,
7,Frozen waffles,


In [25]:
cost = m.addVariable('cost','free',description="total cost")
x = m.addVariable('x','integer',domain = Item,description="number of servings for each item")


In [26]:

# Define inflation rates
optimistic_rate = 0.015  # 1.5%
current_rate = 0.024    # 2.4%
less_optimistic_rate = 0.035  # 3.5%

# Function to apply inflation and create adjusted prices
def adjust_prices(df, inflation_rate):
    # Format the inflation rate as a percentage string
    rate_percent = f"{inflation_rate*100:.1f}%"  # Formats to one decimal place
    df[f'Price 2024 ({rate_percent})'] = df['Price'] * (1 + inflation_rate)
    return df

# Apply the function for each inflation rate
df = adjust_prices(df, optimistic_rate)
df = adjust_prices(df, current_rate)
df = adjust_prices(df, less_optimistic_rate)

# Print column names to verify
print(df.columns)

# Function to generate price distribution
def generate_price_distribution(price):
    np.random.seed(42)  # For reproducibility
    price_variation_factor = 0.05  # 5% variation for the distribution
    return np.random.normal(price, price * price_variation_factor, 1000)

# Apply the function to generate distributions for 2024 prices for each scenario
df['Price Distribution 2024 (Optimistic)'] = df['Price 2024 (1.5%)'].apply(generate_price_distribution)
df['Price Distribution 2024 (Current)'] = df['Price 2024 (2.4%)'].apply(generate_price_distribution)
df['Price Distribution 2024 (Less Optimistic)'] = df['Price 2024 (3.5%)'].apply(generate_price_distribution)


Index(['Calories', 'Serving', 'Protein', 'Carbs', 'Fat', 'Calcium', 'Price',
       'Price 2024 (1.5%)', 'Price 2024 (2.4%)', 'Price 2024 (3.5%)'],
      dtype='object')


In [29]:
%%gams
scalar
    requirements /3000/,
    time_multiplier /7/,
    strictBudget /70/;  # Your strict budget
    
parameter
    minProportion /0.2/  
    maxProportion /0.8/;
    
parameter currentBudgetProportions(foodGroup);
if (strictBudget <= 50,  
    currentBudgetProportions(foodGroup) = budgetScenarios("lowBudget", foodGroup);
else if (strictBudget > 50, 
    currentBudgetProportions(foodGroup) = budgetScenarios("highBudget", foodGroup);

parameter budgetScenarios(scenario, foodGroup) /
    "lowBudget.meat" 0.15, "lowBudget.fruit" 0.1, "lowBudget.vegetable" 0.1, "lowBudget.dairy" 0.1, "lowBudget.snack" 0.25, "lowBudget.breakfast" 0.1, "lowBudget.drink" 0.1, "lowBudget.frozen" 0.1,
    "highBudget.meat" 0.3, "highBudget.fruit" 0.1, "highBudget.vegetable" 0.25, "highBudget.dairy" 0.05, "highBudget.snack" 0.05, "highBudget.breakfast" 0.1, "highBudget.drink" 0.1, "highBudget.frozen" 0.05
/;



    
alias(Item, i);
    
equations
    obj "minimize total meal cost"
    cal_constraint_up
    cal_constraint_low
    protein_constraint_up
    protein_constraint_low
    carb_constraint_up
    carb_constraint_low
    fat_constraint_up
    fat_constraint_low
    calcium_constraint_up
    calcium_constraint_low
    cost_constraint
    cost_constraint_low
    meat_constraint_up
    meat_constraint_low
    fruit_constraint_up
    fruit_constraint_low
    veg_constraint_up
    veg_constraint_low
    breakfast_constraint_up
    breakfast_constraint_low
    frozen_constraint_up
    frozen_constraint_low
    drink_constraint_up
    drink_constraint_low
    dessert_constraint_up
    dessert_constraint_low
    dry_constraint_up
    dry_constraint_low
    snack_up
    snack_lo
    dairy_constraint_up
    dairy_constraint_low;
    
obj..
    cost =e= sum(Item, price(Item)*x(Item));

# Do not go over budget
cost_constraint..
    cost =l= 150;
    
cost_constraint_low..
    cost =g= 70;

# Make sure we meet our nutritional goals
cal_constraint_up..
    sum(Item, x(Item) * calories(Item)) =l= 3000 * time_multiplier;  # Upper calorie limit
    
cal_constraint_low..
    sum(Item, x(Item) * calories(Item)) =g= 2500 * time_multiplier;  # Lower calorie limit
    
protein_constraint_up..
    sum(i, x(i) * protein(i)) =l= 56 * time_multiplier;  # Upper protein limit
    
protein_constraint_low..
    sum(i, x(i) * protein(i)) =g= 46 * time_multiplier;  # Lower protein limit
    
carb_constraint_up..
    sum(i, x(i) * carbs(i)) =l= 375 * time_multiplier;  # Upper carb limit
    
carb_constraint_low..
    sum(i, x(i) * carbs(i)) =g= 313 * time_multiplier;  # Lower carb limit
    
fat_constraint_up..
    sum(i, x(i) * fat(i)) =l= 100 * time_multiplier;  # Upper fat limit
    
fat_constraint_low..
    sum(i, x(i) * fat(i)) =g= 83 * time_multiplier;  # Lower fat limit
    
calcium_constraint_up..
    sum(i, x(i) * calcium(i)) =l= 1100 * time_multiplier;  # Upper calcium limit
    
calcium_constraint_low..
    sum(i, x(i) * calcium(i)) =g= 900 * time_multiplier;  # Lower calcium limit

    
# Make sure we diversify the food groups we are eating   
    
meat_constraint_up..
    sum(meat, price(meat) * x(meat)) =l= strictBudget * currentBudgetProportions("meat") * maxProportion;

meat_constraint_low..
    sum(meat, price(meat) * x(meat)) =g= strictBudget * currentBudgetProportions("meat") * minProportion;

fruit_constraint_up..
    sum(fruit, price(fruit) * x(fruit)) =l= strictBudget * currentBudgetProportions("fruit") * maxProportion;
    
fruit_constraint_low..
   sum(fruit, price(fruit) * x(fruit)) =g= strictBudget * currentBudgetProportions("fruit") * minProportion;
    
veg_constraint_up..
   sum(vegetable, price(vegetable) * x(vegetable)) =l= strictBudget * currentBudgetProportions("vegetables") * maxProportion ;
    
veg_constraint_low..
    sum(vegetable, price(vegetable) * x(vegetable)) =g= strictBudget * currentBudgetProportions("vegetables") * minProportion;
    
breakfast_constraint_up..
    sum(breakfast, price(breakfast) * x(breakfast)) =l= strictBudget * currentBudgetProportions("breakfast") * maxProportion;
    
breakfast_constraint_low..
    sum(breakfast, price(breakfast) * x(breakfast)) =g= strictBudget * currentBudgetProportions("breakfast") * minProportion;
    
frozen_constraint_up..
    sum(frozen, price(frozen) * x(frozen)) =l= strictBudget * currentBudgetProportions("frozen") * maxProportion;
    
frozen_constraint_low..
    sum(frozen, price(frozen) * x(frozen)) =g= strictBudget * currentBudgetProportions("frozen") * minProportion;
    
drink_constraint_up..
    sum(drink, price(drink) * x(drink)) =l= strictBudget * currentBudgetProportions("drink") * maxProportion;

    
drink_constraint_low..
    sum(drink, price(drink) * x(drink)) =g= strictBudget * currentBudgetProportions("drink") * minProportion;
    
dessert_constraint_up..
    sum(dessert, price(dessert) * x(dessert)) =l= strictBudget * currentBudgetProportions("dessert") * maxProportion;
    
dessert_constraint_low..
    sum(dessert, price(dessert) * x(dessert)) =g= strictBudget * currentBudgetProportions("dessert") * minProportion;
    
dry_constraint_up..
    sum(dry, price(dry) * x(dry)) =l= strictBudget * currentBudgetProportions("dry") * maxProportion;
    
dry_constraint_low..
    sum(dry, price(dry) * x(dry)) =g= strictBudget * currentBudgetProportions("dry") * minProportion;
    
snack_up..
     sum(snack, price(snack) * x(snack) =l= strictBudget * currentBudgetProportions("snack") * maxProportion;
    
snack_lo..
     sum(snack, price(snack) * x(snack) =g= strictBudget * currentBudgetProportions("snack") * minProportion;
    
dairy_constraint_up..
     sum(dairy, price(dairy) * x(dairy) =l= strictBudget * currentBudgetProportions("dairy") * maxProportion;
    
dairy_constraint_low..
     sum(dairy, price(dairy) * x(dairy) =g= strictBudget * currentBudgetProportions("dairy") * minProportion;
    
    
# X cannot be negative
x.lo(i) = 0;
 
model plan /obj 
    cal_constraint_up
    cal_constraint_low
    protein_constraint_up
    protein_constraint_low
    carb_constraint_up
    carb_constraint_low
    fat_constraint_up
    fat_constraint_low
    calcium_constraint_up
    calcium_constraint_low
    cost_constraint
    cost_constraint_low
    meat_constraint_up
    meat_constraint_low
    fruit_constraint_up
    fruit_constraint_low
    veg_constraint_up
    veg_constraint_low
    breakfast_constraint_up
    breakfast_constraint_low
    frozen_constraint_up
    frozen_constraint_low
    drink_constraint_up
    drink_constraint_low
    dessert_constraint_up
    dessert_constraint_low
    dry_constraint_up
    dry_constraint_low
    snack_up
    snack_lo
    dairy_constraint_up
    dairy_constraint_low
    /;
solve plan using mip minimizing cost;



GamsExceptionExecution: GAMS return code not 0 (2), check /home/user004/gj_327180f0_base_4.lst for more details

In [ ]:
display(x.records)

In [ ]:
display(x.records[x.records["level"] !=0])